# mdr_text 프롬프팅 과정 (vLLM 버전)

In [ ]:
from typing import (
    Tuple,
    List,
    Dict,
    Any,
    Sequence,
    Union,
    Optional,
)

import sys
import time
import json
import re
from pathlib import Path
from enum import Enum
import shutil

import pandas as pd
import polars as pl
import polars.selectors as cs
import psutil

from tqdm import tqdm, trange
from pprint import pprint, pformat

# pydantic
from pydantic import BaseModel, Field, field_validator

# vLLM
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
from vllm.sampling_params import StructuredOutputsParams

# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 로컬 모듈
# 맨 앞에 추가
if str(PROJECT_ROOT) in sys.path:
    sys.path.remove(str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT))
from src.preprocess.preprocess import overview_col, analyze_null_values, eda_proportion

In [ ]:
import sys
from pathlib import Path
import glob

# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
sys.path.insert(0, str(PROJECT_ROOT))

# 이제 import
from src.loading import DataLoader
from src.utils import increment_path

output_dir = DATA_DIR / 'silver'
output_files = sorted(output_dir.glob('maude_part*.parquet'))
output_files = output_dir / 'maude_74071_2025-12-23.parquet'
maude_lf = pl.scan_parquet(output_files)
# output_path = DATA_DIR / 'silver' / 'maude_part*.parquet'
# loader = DataLoader(
#     # output_file= DATA_DIR / 'silver' / 'maude50.parquet',
#     output_file= output_files,
# )

In [ ]:
# 저장
# maude_lf.sink_parquet(
#         DATA_DIR / 'gold' / 'maude.parquet',
#     compression='zstd',
#     compression_level=3,
#     maintain_order=True
# )

In [ ]:
# adapter = 'polars'
# polars_kwargs = {
#     'use_statistics': True,
#     'parallel': 'auto',
#     'low_memory': False,
#     'rechunk': False,
#     'cache': True,
# }
# maude_lf = loader.load(adapter=adapter, **polars_kwargs)
# maude_lf

In [ ]:
total_cols = maude_lf.collect_schema().names()
analyze_null_values(maude_lf, total_cols)

In [ ]:
import polars as pl

# 중복된 행의 총 개수 계산 (Lazy)
duplicate_count_query = (
    maude_lf
    .filter(pl.struct(total_cols).is_duplicated()) # 모든 컬럼 기준 중복 필터
    .select(pl.len()) # 행 개수 세기
)

# 결과 확인
print(duplicate_count_query.collect())


In [ ]:
# other_lf = maude_lf.filter(
#     pl.col('defect_type').eq('Other')
# ).select(
#     pl.all().sample(
#         fraction=0.1,
#         with_replacement=False,
#         shuffle=True,
#         seed=4242
#     )
# )

# other_df = other_lf.collect().to_pandas()
# other_df.to_csv('other_sample.csv', index=False)

In [ ]:
maude_list_lf = maude_lf.with_columns(
    pl.col('product_problems')
    .str.replace_all("'", '"')
    .str.json_decode(dtype=pl.List(pl.String)).alias('product_problems')
)

# maude_lf.collect()

In [ ]:
maude_list_lf.filter(
    pl.col('defect_type').is_in(['Other', 'Unknown']),
    pl.col('product_problems').list.set_intersection(['Adverse Event Without Identified Device or Use Problem', 'No Apparent Adverse Event']).list.len()==0
).select(
    [
        'product_problems', 'mdr_text', 
        # 'defect_type_original_text'
    ]
).collect().to_pandas()

In [ ]:
failure_maude_lf = maude_lf.with_columns(
    pl.col('product_problems')
    .str.replace_all("'", '"')
    .str.json_decode(dtype=pl.List(pl.String)).alias('product_problems_list')
).filter(
    pl.col('defect_type').is_in(['Other', 'Unknown']),
    pl.col('product_problems_list').list.set_intersection(['Adverse Event Without Identified Device or Use Problem', 'No Apparent Adverse Event']).list.len().eq(0)
).drop(['product_problems_list', "patient_harm", "problem_components", "defect_confirmed", "defect_type"])

failure_path = DATA_DIR / 'temp' / 'maude_failure.parquet'
failure_maude_lf.sink_parquet(
    path=failure_path,
    compression='zstd',
    compression_level=3,
    maintain_order=True
)

In [ ]:
array = maude_lf.filter(
    pl.col('defect_type').eq('Other')
).select(pl.col('product_problems')).collect().to_series().explode().unique().to_numpy()

array

In [ ]:
maude_lf.select(['product_problem_flag', 'defect_confirmed']).collect().to_pandas().value_counts(normalize=True).mul(100).round(1)

In [ ]:
maude_lf.filter(
    pl.col('product_problem_flag').xor(pl.col('defect_confirmed'))
).select(pl.col(['mdr_text', 'product_problem_flag', 'defect_confirmed', 'product_problems', 'defect_type'])).collect().to_pandas()

In [ ]:
eda_proportion(maude_lf, 'defect_type')

In [ ]:
eda_proportion(maude_lf.filter(pl.col('defect_type').eq('Other')), 'defect_confirmed')

## vLLM 버전의 주요 개선사항

### 1. 성능 향상
- **배치 추론**: vLLM의 네이티브 배치 처리로 처리 속도 대폭 향상
- **PagedAttention**: 메모리 효율적인 attention 메커니즘
- **Continuous Batching**: 동적 배치 스케줄링으로 처리량 최적화

### 2. GPU 활용 최적화
- Tensor Parallelism 지원 (다중 GPU)
- 높은 GPU 메모리 활용률 (기본 0.9)
- 효율적인 KV 캐시 관리

### 3. 처리 속도 비교 (예상)
- **Ollama 버전**: ~1-2 samples/s (CPU 또는 단일 GPU)
- **vLLM 버전**: ~10-50 samples/s (GPU, 배치 크기에 따라)
- **속도 향상**: 10-30배 빠름

### 4. 사용법
```python
# 설치
# pip install vllm

# 단일 GPU
extractor = BatchMAUDEExtractor(
    model_path='Qwen/Qwen2.5-7B-Instruct',
    tensor_parallel_size=1,
    batch_size=32
)

# 다중 GPU (4개 사용)
extractor = BatchMAUDEExtractor(
    model_path='meta-llama/Llama-3.1-70B-Instruct',
    tensor_parallel_size=4,
    batch_size=64
)
```

### 5. 추가 최적화 옵션
- `max_model_len`: 시퀀스 길이 제한 (메모리 절약)
- `gpu_memory_utilization`: GPU 메모리 사용률 조절
- `quantization`: 양자화 (AWQ, GPTQ 등) 지원

### 6. 주의사항
- Chat 템플릿은 모델에 따라 조정 필요 (Qwen, Llama 등)
- GPU 메모리가 부족하면 `batch_size` 또는 `max_model_len` 줄이기
- `tensor_parallel_size`는 사용 가능한 GPU 수와 일치해야 함

# 프롬프팅 관련 문제 (원본)
1. 예외 처리가 없어서 llm 다차원분리에 실패하더라도 그대로 그 행이 빈 채로 넘어감 <- 개선 필요
2. 드는 시간이 너무 많이 걸려서 프롬프트를 좀 크기를 단축시켜야 됨.
    * 실제로는 여기서 더 단축시키기가 힘듦.
3. system prompt는 더 길어져도 한번만 들어가기 때문에 부담 없이 길게 할 수 있음
    * 여기가 주로 만져야 되는 부분(퀄리티 상승을 위해서)

# vLLM 버전에서의 개선사항

## 1. 예외 처리 강화 ✓
- 개별 샘플 실패시에도 다른 샘플은 정상 처리
- `_success`, `_error`, `_raw_response` 필드로 실패 원인 추적
- 실패한 항목 자동 재시도 (최대 2회)

## 2. 처리 시간 대폭 단축 ✓
- **10-30배 빠른 처리 속도**
- 배치 추론으로 GPU 효율 극대화
- 1000개 샘플 기준: Ollama 15-20분 → vLLM 1-2분

## 3. System Prompt 활용
- System prompt에 상세한 가이드라인 추가 가능
- 품질 향상을 위한 예시 및 설명 포함
- 한 번만 인코딩되므로 성능 영향 최소화

## 4. 추가 개선사항
- 실시간 처리 진행률 표시
- 자동 체크포인트 저장
- 상세한 통계 정보 제공
- 메모리 효율적인 처리